# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

02-15-2023


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-02-16 04:20:56,33.93911,67.709953,209056,7896,NaN,NaN,Afghanistan,537.027766,3.776978
1,NaN,NaN,NaN,Albania,2023-02-16 04:20:56,41.15330,20.168300,334264,3596,NaN,NaN,Albania,11615.261658,1.075796
2,NaN,NaN,NaN,Algeria,2023-02-16 04:20:56,28.03390,1.659600,271421,6881,NaN,NaN,Algeria,618.961332,2.535176
3,NaN,NaN,NaN,Andorra,2023-02-16 04:20:56,42.50630,1.521800,47860,165,NaN,NaN,Andorra,61942.664855,0.344756
4,NaN,NaN,NaN,Angola,2023-02-16 04:20:56,-11.20270,17.873900,105184,1931,NaN,NaN,Angola,320.036336,1.835831


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/6/23,2/7/23,2/8/23,2/9/23,2/10/23,2/11/23,2/12/23,2/13/23,2/14/23,2/15/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,208704,208721,208771,208771,208943,208971,208982,209011,209036,209056
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334211,334211,334222,334229,334229,334234,334255,334255,334264,334264
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271395,271399,271403,271406,271406,271409,271409,271409,271409,271421


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/6/23,2/7/23,2/8/23,2/9/23,2/10/23,2/11/23,2/12/23,2/13/23,2/14/23,2/15/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/6/23,2/7/23,2/8/23,2/9/23,2/10/23,2/11/23,2/12/23,2/13/23,2/14/23,2/15/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/6/23,2/7/23,2/8/23,2/9/23,2/10/23,2/11/23,2/12/23,2/13/23,2/14/23,2/15/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19530,19530,19630,19630,19630,19630,19630,19630,19630,19686
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,69187,69187,69344,69344,69344,69344,69344,69344,69344,69521
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7339,7339,7387,7387,7387,7387,7387,7387,7387,7431


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/6/23,2/7/23,2/8/23,2/9/23,2/10/23,2/11/23,2/12/23,2/13/23,2/14/23,2/15/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,230,230,230,230,230,230,230,230,230,230
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,723,723,723,723,723,723,723,723,723,724
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,103,103,103,103,103,103,103,103,103,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2023-02-16 04:20:56,32.539527,-86.644082,19686,230,NaN,NaN,"Autauga, Alabama, US",35235.998496,1.168343
715,1075.0,Lamar,Alabama,US,2023-02-16 04:20:56,33.779950,-88.096680,4703,73,NaN,NaN,"Lamar, Alabama, US",34067.366896,1.552201
716,1077.0,Lauderdale,Alabama,US,2023-02-16 04:20:56,34.901719,-87.656247,30238,419,NaN,NaN,"Lauderdale, Alabama, US",32609.000421,1.385674


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,102995115,1115564,0.0
India,44684376,530756,0.0
France,39779373,165698,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,102995115,1115564,0.0,101879551.0,2023-02-16 04:20:56,37.934269,-91.444083
India,44684376,530756,0.0,44153620.0,2023-02-16 04:20:56,23.088275,81.806127
France,39779373,165698,0.0,39613675.0,2023-02-16 04:20:56,6.430808,-34.730285
Germany,37970357,167124,0.0,37803233.0,2023-02-16 04:20:56,51.571844,10.277881
Brazil,36971440,697904,0.0,36273536.0,2023-02-16 04:20:56,-12.669522,-48.480493


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,12031907,100314,0.0
Texas,8416861,93040,0.0
Florida,7483857,85710,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,12031907,100314,0.0,11931593.0,2023-02-16 04:20:56,37.843962,-120.728594
Texas,8416861,93040,0.0,8323821.0,2023-02-16 04:20:56,31.660643,-98.653069
Florida,7483857,85710,0.0,7398147.0,2023-02-16 04:20:56,28.940755,-82.700744
New York,6754579,76904,0.0,6677675.0,2023-02-16 04:20:56,42.544151,-75.474183
Illinois,4040139,41261,0.0,3998878.0,2023-02-16 04:20:56,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3684071,35132,0.0
Florida,Miami-Dade,1532574,12049,0.0
Illinois,Cook,1515818,15213,0.0
Arizona,Maricopa,1499125,18756,0.0
Texas,Harris,1268181,11578,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3684071,35132,0.0,3648939.0,2023-02-16 04:20:56,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1532574,12049,0.0,1520525.0,2023-02-16 04:20:56,25.611236,-80.551706,12086.0
Illinois,Cook,1515818,15213,0.0,1500605.0,2023-02-16 04:20:56,41.841448,-87.816588,17031.0
Arizona,Maricopa,1499125,18756,0.0,1480369.0,2023-02-16 04:20:56,33.348359,-112.491815,4013.0
Texas,Harris,1268181,11578,0.0,1256603.0,2023-02-16 04:20:56,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,209011,334255,271409,47860,105184,11,9106,10042136,446332,11330342,...,1033265,250799,12014,551801,11526704,703228,535,11945,342317,263083
2023-02-14,209036,334264,271409,47860,105184,11,9106,10042136,446332,11330342,...,1033766,250799,12014,551804,11526754,703228,535,11945,342317,263083
2023-02-15,209056,334264,271421,47860,105184,11,9106,10042136,446332,11330342,...,1033766,250829,12014,551874,11526773,703228,535,11945,342317,263642


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,7896,3596,6881,165,1931,0,146,130448,8719,19070,...,7614,1637,14,5851,43186,5708,0,2159,4051,5659
2023-02-14,7896,3596,6881,165,1931,0,146,130448,8719,19070,...,7617,1637,14,5851,43186,5708,0,2159,4051,5659
2023-02-15,7896,3596,6881,165,1931,0,146,130448,8719,19070,...,7617,1637,14,5851,43186,5708,0,2159,4051,5662


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/6/23,2/7/23,2/8/23,2/9/23,2/10/23,2/11/23,2/12/23,2/13/23,2/14/23,2/15/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19530,19530,19630,19630,19630,19630,19630,19630,19630,19686
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4675,4675,4689,4689,4689,4689,4689,4689,4689,4703
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,30057,30057,30136,30136,30136,30136,30136,30136,30136,30238
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,9323,9323,9340,9340,9340,9340,9340,9340,9340,9353
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,46938,46938,47170,47170,47170,47170,47170,47170,47170,47374


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-02-13,1627670,305060,8320,2404386,999652,12031907,1753754,971466,327873,49,...,2496032,8400059,1083959,150985,24685,2269006,1915286,637762,1991319,184251
2023-02-14,1627670,305846,8320,2404386,999965,12031907,1754809,971634,327873,49,...,2496032,8401082,1083959,150985,24690,2276356,1915286,638249,1992070,184505
2023-02-15,1633743,305846,8320,2404386,1000333,12031907,1754830,972133,327873,49,...,2496032,8416861,1083959,151477,24690,2276356,1915286,638456,1992853,184505


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2023-02-13       19630   69344    7387  8012  18488    3002   6478   40979   
2023-02-14       19630   69344    7387  8012  18488    3002   6478   40979   
2023-02-15       19686   69521    7431  8049  18554    3017   6503   41101   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-02-13        10746     6701  ...    7663   2256     9838     2306   
2023-02-14        10746     6701  ...    7677   2257     9884     2306   
2023-02-15        10776     6725  ...    7677   2257     9884     2306   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2023-02-13          12468  12104  6368          0     2741   1888  
2023-02-14          12476  12113  6373          0     2744   1897  
2023-02-15          12476  12113  6373          0     2744   1897  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-02-13,20892,1473,34,32936,12925,100300,14083,12036,3288,0,...,29113,92919,5270,901,130,23462,15510,7910,16217,1994
2023-02-14,20892,1473,34,32936,12930,100300,14083,12036,3288,0,...,29113,92920,5270,901,130,23599,15510,7913,16239,1996
2023-02-15,20899,1473,34,32936,12936,100314,14105,12061,3288,0,...,29113,93040,5270,910,130,23599,15510,7606,16239,1996


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2023-02-13         230     723     103  109    261      54    132     674   
2023-02-14         230     723     103  109    261      54    132     674   
2023-02-15         230     724     103  109    261      54    132     674   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-02-13          170       89  ...     153     44       90       28   
2023-02-14          170       89  ...     153     44       90       28   
2023-02-15          170       89  ...     153     44       90       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-02-13            138    16    43          0       50     23  
2023-02-14            138    16    43          0       50     23  
2023-02-15            138    16    43          0       50     23  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,0.000139,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00018,0.0,0.0,...,0.000000,0.000223,0.0,0.000147,0.000001,0.0,0.0,0.0,0.000085,0.000000
2023-02-14,0.000120,0.000027,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000485,0.000000,0.0,0.000005,0.000004,0.0,0.0,0.0,0.000000,0.000000
2023-02-15,0.000096,0.000000,0.000044,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.000000,0.000120,0.0,0.000127,0.000002,0.0,0.0,0.0,0.000000,0.002125


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000084,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.00000
2023-02-14,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.0,0.0,...,0.000394,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.00000
2023-02-15,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.00053


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-02-13,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.001067,0.000134,0.0,0.0,...,0.0,0.000078,0.0,0.000000,0.000527,0.000000,0.0,0.001039,0.000674,0.000000
2023-02-14,0.000000,0.002576,0.0,0.0,0.000313,0.0,0.000602,0.000173,0.0,0.0,...,0.0,0.000122,0.0,0.000000,0.000203,0.003239,0.0,0.000764,0.000377,0.001379
2023-02-15,0.003731,0.000000,0.0,0.0,0.000368,0.0,0.000012,0.000514,0.0,0.0,...,0.0,0.001878,0.0,0.003259,0.000000,0.000000,0.0,0.000324,0.000393,0.000000


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin   Barbour      Bibb   Blount   Bullock   
2023-02-13      0.000000  0.000000  0.000000  0.000000  0.00000  0.000000   
2023-02-14      0.000000  0.000000  0.000000  0.000000  0.00000  0.000000   
2023-02-15      0.002853  0.002553  0.005956  0.004618  0.00357  0.004997   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2023-02-13      0.000000  0.000000  0.000000  0.000000  ...  0.000000   
2023-02-14      0.000000  0.000000  0.000000  0.000000  ...  0.001827   
2023-02-15      0.003859  0.002977  0.002792  0.003582  ...  0.000000   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2023-02-13      0.000000  0.000000      0.0   0.000000  0.000000  0.000000   
2023-02-14      0.000443  0.004676      0.0   0.000642  0.000744  0.000785   
2023-02-15      0.000000  0.000000      0.0   0.000000  0.000000  0.000000   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2023-02-13            NaN  0.000000  0.000000  
2023-02-14            NaN  0.001094  0.004767  
2023-02-15            NaN  0.000000  0.000000  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-02-13,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,NaN,...,0.0,0.000118,0.0,0.000000,0.007752,0.000000,0.0,0.000759,0.001482,0.000000
2023-02-14,0.000000,0.0,0.0,0.0,0.000387,0.00000,0.000000,0.000000,0.0,NaN,...,0.0,0.000011,0.0,0.000000,0.000000,0.005839,0.0,0.000379,0.001357,0.001003
2023-02-15,0.000335,0.0,0.0,0.0,0.000464,0.00014,0.001562,0.002077,0.0,NaN,...,0.0,0.001291,0.0,0.009989,0.000000,0.000000,0.0,-0.038797,0.000000,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2023-02-13         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2023-02-14         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2023-02-15         0.0  0.001383     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-02-13          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2023-02-14          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2023-02-15          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-02-13            0.0   0.0   0.0        NaN      0.0    0.0  
2023-02-14            0.0   0.0   0.0        NaN      0.0    0.0  
2023-02-15            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,0.000157,0.000019,0.000002,0.000007,0.0,0.0,0.0,0.000091,0.000012,0.000055,...,0.000002,0.000122,0.0,0.000096,0.000002,0.0,0.0,0.0,0.000136,0.000046
2023-02-14,0.000138,0.000023,0.000001,0.000003,0.0,0.0,0.0,0.000046,0.000006,0.000028,...,0.000244,0.000061,0.0,0.000051,0.000003,0.0,0.0,0.0,0.000068,0.000023
2023-02-15,0.000117,0.000011,0.000023,0.000002,0.0,0.0,0.0,0.000023,0.000003,0.000014,...,0.000122,0.000090,0.0,0.000089,0.000002,0.0,0.0,0.0,0.000034,0.001074


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,1.200000e-06,0.0,0.0,0.0,0.0,NaN,0.0,0.000043,3.600000e-06,0.000416,...,0.000003,0.0,0.0,0.000006,0.0,0.0,NaN,0.0,0.000039,0.000003
2023-02-14,6.000000e-07,0.0,0.0,0.0,0.0,NaN,0.0,0.000021,1.800000e-06,0.000208,...,0.000198,0.0,0.0,0.000003,0.0,0.0,NaN,0.0,0.000019,0.000001
2023-02-15,3.000000e-07,0.0,0.0,0.0,0.0,NaN,0.0,0.000011,9.000000e-07,0.000104,...,0.000099,0.0,0.0,0.000002,0.0,0.0,NaN,0.0,0.000010,0.000266


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-02-13,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-02-14,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-02-15,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-02-13,0.000095,0.000018,0.0,0.000020,0.000130,0.000051,0.000543,0.000285,0.000049,0.0,...,0.000102,0.000133,0.00004,0.000033,0.000354,0.000029,0.000046,0.000573,0.000383,0.000046
2023-02-14,0.000048,0.001297,0.0,0.000010,0.000222,0.000026,0.000572,0.000229,0.000025,0.0,...,0.000051,0.000127,0.00002,0.000017,0.000278,0.001634,0.000023,0.000668,0.000380,0.000712
2023-02-15,0.001889,0.000649,0.0,0.000005,0.000295,0.000013,0.000292,0.000371,0.000012,0.0,...,0.000025,0.001003,0.00001,0.001638,0.000139,0.000817,0.000012,0.000496,0.000387,0.000356


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2023-02-13      0.000080  0.000036  0.000103  0.000089  0.000119  0.000058   
2023-02-14      0.000040  0.000018  0.000051  0.000044  0.000060  0.000029   
2023-02-15      0.001447  0.001285  0.003004  0.002331  0.001815  0.002513   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2023-02-13      0.000071  0.000064  0.000060  0.000031  ...  0.000037   
2023-02-14      0.000036  0.000032  0.000030  0.000015  ...  0.000932   
2023-02-15      0.001947  0.001505  0.001411  0.001798  ...  0.000466   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2023-02-13      0.000066  0.000043  0.000027   0.000073  0.000030  0.000063   
2023-02-14      0.000255  0.002359  0.000013   0.000358  0.000387  0.000424   
2023-02-15      0.000127  0.001180  0.000007   0.000179  0.000193  0.000212   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2023-02-13      -0.503937  0.000034  0.000033  
2023-02-14      -0.503937  0.000564  0.002400  
2023-02-15      -0.503937  0.000282  0.001200  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-02-13,0.000008,0.0,0.0,0.000026,0.000225,0.000060,0.000039,0.000081,0.000111,NaN,...,0.000042,0.000120,0.000066,0.000123,0.003876,0.000123,0.000077,0.000493,0.000833,0.000059
2023-02-14,0.000004,0.0,0.0,0.000013,0.000306,0.000030,0.000019,0.000041,0.000055,NaN,...,0.000021,0.000066,0.000033,0.000062,0.001938,0.002981,0.000038,0.000436,0.001095,0.000531
2023-02-15,0.000169,0.0,0.0,0.000006,0.000385,0.000085,0.000791,0.001059,0.000028,NaN,...,0.000010,0.000678,0.000016,0.005025,0.000969,0.001491,0.000019,-0.019180,0.000547,0.000266


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock    Butler   
2023-02-13         0.0  0.000000     0.0  0.0    0.0     0.0  0.000119   
2023-02-14         0.0  0.000000     0.0  0.0    0.0     0.0  0.000060   
2023-02-15         0.0  0.000692     0.0  0.0    0.0     0.0  0.000030   

Province_State                            ...   Wyoming                   \
Admin2         Calhoun Chambers Cherokee  ...      Park Platte  Sheridan   
2023-02-13         0.0      0.0      0.0  ...  0.000051    0.0  0.000178   
2023-02-14         0.0      0.0      0.0  ...  0.000026    0.0  0.000089   
2023-02-15         0.0      0.0      0.0  ...  0.000013    0.0  0.000045   

Province_State                                                                 
Admin2         Sublette Sweetwater Teton Uinta Unassigned  Washakie    Weston  
2023-02-13          0.0   0.000057   0.0   0.0       -1.0  0.000499  0.000355  
2023-02-14          0.0   0.000029   0.0   0.0       -1.0  0.000249  0.000178  
2023-02-15          0.0   0.000014   0.0   0.0       -1.0  0.000125  0.000089  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 